In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Thu May 28 07:25:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   40C    P8     9W / 250W |     10MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:09:00.0 Off |                  N/A |
| 40%   68C    P2   144W / 250W |  11213MiB / 12196MiB |     51%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [4]:
import sys
import time
from collections import defaultdict
import numpy as onp

# PyTorch

In [5]:
import torch
import modules.pytorch_models

In [7]:
model = modules.pytorch_models.AlexNet(10)

In [8]:
def compute_output(inp, k, s, p):
    out = int(np.floor((inp - k + 2*p)/s) + 1)
    return out

In [17]:
img_dim = 224
X = torch.zeros((10, 3, img_dim, img_dim))
out = model(X)
out.shape

Feature Stack
x:      (10, 3, 224, 224)
conv1:  (10, 64, 55, 55)
mp1:    (10, 64, 27, 27)
conv2:  (10, 192, 27, 27)
mp2:    (10, 192, 13, 13)
conv3:  (10, 384, 13, 13)
conv4:  (10, 256, 13, 13)
conv5:  (10, 256, 13, 13)
mp3:    (10, 256, 6, 6)
PostAP: (10, 256, 6, 6)
Flat:   (10, 9216)
Classifier Stack


torch.Size([10, 10])

# JAX

In [ ]:
import jax.numpy as np
from jax import jit, grad, random
import modules.jax_models

img_format = 'NCHW'

rng = onp.random.RandomState(0)
rng_key = random.PRNGKey(0)

batch_size = 2
input_shape = (batch_size, 3, 224, 224)

In [ ]:
# Fake image
images = rng.rand(*input_shape).astype('float32')
images.shape

In [ ]:
init, apply = modules.jax_models.resnet.ResNet152(num_classes=10, img_fmt=img_format, img_dim=input_shape[2:])
_, params = init(rng_key, input_shape)

In [ ]:
out = apply(params, images)
out.shape